# CD ROM control

###open and close cd rom






In [ ]:
import ctypes

#open cdrom
ctypes.windll.WINMM.mciSendStringW(u"set cdaudio door ", None, 0, None)

#close cdrom
ctypes.windll.WINMM.mciSendStringW(u"set cdaudio door closed", None, 0, None)

### moniter cd rom with IMPAI

In [ ]:
class Sink(object):
    def DDiscFormat2DataEvents_Update(self, this, progress):
        strTimeStatus = 'Time: ' + progress.ElapsedTime + ' / ' + progress.TotalTime
        print(strTimeStatus)

import comtypes.client as cc

class MyProgram(object):
    def main(self):
        # index of recording drive
        index = 0

        # files to transfer to disc
        path = r'D:\Example'

        # DiscMaster2 object to connect to optical drives
        discMaster = cc.CreateObject('IMAPI2.MsftDiscMaster2')
        # cc.ShowEvents(discMaster) ### uncomment for print events

        # DiscRecorder object for specified burning device
        recorder = cc.CreateObject('IMAPI2.MsftDiscRecorder2')
        uniqueId = discMaster.Item(index)
        recorder.InitializeDiscRecorder(uniqueId)

        # create an image stream for a specified directory
        # create a new file system image and retrieve root directory
        _fsi = cc.CreateObject('IMAPI2FS.MsftFileSystemImage')
        _dir = _fsi.Root
        _fsi.VolumeName = 'TestName'

        # create the new disc format and set the recorder
        dataWriter = cc.CreateObject('IMAPI2.MsftDiscFormat2Data')
        # cc.ShowEvents(dataWriter) ### uncomment for print events
        dataWriter.Recorder = recorder
        dataWriter.ClientName = 'MyClient'
        dataWriter.ForceOverwrite = True

        _fsi.ChooseImageDefaults(recorder)

        # get events
        sink = Sink()
        _events = cc.GetEvents(dataWriter, sink)
        # print(_events)  ### <comtypes.client._events._AdviseConnection object at 0x...>

        # add the directory and its content to the file system
        _dir.AddTree(path, False)

        # create an image from the file system
        _result = _fsi.CreateResultImage()
        _stream = _result.ImageStream

        # write disc using the specified recorder
        print('Writing content to disc...')
        dataWriter.Write(_stream)
        print('Finished writing content')

### list the cd rom device

In [ ]:
import wmi
c = wmi.WMI()
for cdrom in c.Win32_CDROMDrive():
    print cdrom.Drive, cdrom.MediaLoaded

### canceling the write operation.

In [ ]:
import imapi2

# update the cd rom status.
DDiscFormat2EraseEvents.Update

#cancel the current write operation.
IDiscFormat2RawCD.CancelWrite



###notify device added

In [ ]:
import sink as Sink
import comtypes.client as cc
from comtypes.gen import IMAPI2
from comtypes.gen._2735412F_7F64_5B0F_8F00_5D77AFBE261E_0_1_0 import DDiscMaster2Events, DDiscFormat2DataEvents, \
    IDiscFormat2DataEventArgs, IBurnVerification

IMAPI2.DDiscFormat2EraseEvents


if DDiscMaster2Events.NotifyDeviceAdded is True:
    print("device added")
else:
    print("none")


DDiscFormat2DataEvents.Update     #Implement this method to receive progress notification of the current raw-image write operation.

#IDiscFormat2DataEventArgs.GetTypeInfo( 0,1)    # current write action

#IBurnVerification.get_BurnVerificationLevel


###copy to cd-rom moniter

In [ ]:
import os
import wmi
from comtypes.gen import IMAPI2
from comtypes.gen._2735412F_7F64_5B0F_8F00_5D77AFBE261E_0_1_0 import IDiscFormat2Data, HRESULT

c = wmi.WMI()
for cdrom in c.Win32_CDROMDrive():
    print (cdrom.Drive, cdrom.MediaLoaded)
    if cdrom.MediaLoaded is False:
        print("Media is Loaded")
    #if data matching ALGO loop starts
    #IMAPI2.DDiscFormat2EraseEvents()
    HRESULT.CancelWrite()
    HRESULT.EraseMedia()
    #if data matching ALGO loop ends
